## Imports

In [12]:
import pandas as pd
from plotnine import *
pd.set_option('display.max_columns', None)

## Cluster to Cell-type Mapping

In [13]:
clust_to_celltype = {1: 'excitatory_neurons', 2: 'inhibitory_neurons', 3: 'excitatory_neurons',
                     4: 'excitatory_neurons', 5: 'nigral_neurons', 6: 'nigral_neurons', 7: 'unknown_neurons',
                     8: 'opcs', 9: 'opcs', 10: 'opcs', 11: 'inhibitory_neurons',12: 'inhibitory_neurons',
                     13: 'astrocytes', 14: 'astrocytes', 15: 'astrocytes', 16: 'astrocytes', 17: 'astrocytes',
                     18: 'doublets', 19: 'oligodendrocytes', 20: 'oligodendrocytes', 21: 'oligodendrocytes',
                     22: 'oligodendrocytes', 23: 'oligodendrocytes', 24: 'microglia'}

## Define Scores Dictionary Update Function

In [14]:
def update_dict(snps_dict, snp_row, cluster, celltype, version):
    if version == 'new':
        update_fields = ['effect', 'noneffect', 'explain_pval', 'ism_pval', 'delta_pval',
                         'magnitude_pval', 'prominence_pval', 'confidence']
    elif version == 'old':
        update_fields = ['effect', 'noneffect', 'explain_pval', 'ism_pval', 'delta_pval',
                         'confidence']
    for i in update_fields:
        snps_dict[snp_row['rsid']][i] = snp_row[i]
    snps_dict[snp_row['rsid']]['best_cluster'] = cluster
    snps_dict[snp_row['rsid']]['best_celltype'] = celltype
    if snp_row['confidence'] > -1 and snp_row['effect'] != 'NA' and snp_row['noneffect'] != 'NA':
        if cluster not in snps_dict[snp_row['rsid']]['sig_clusters']:
            snps_dict[snp_row['rsid']]['sig_clusters'].append(cluster)
        if celltype not in snps_dict[snp_row['rsid']]['sig_celltypes']:
            snps_dict[snp_row['rsid']]['sig_celltypes'].append(celltype)

## Define Populate Scores Dictionary Function

In [15]:
def populate_dict(basedir, snps_dict, version):
    for cluster in range(1,25):
        celltype = clust_to_celltype[cluster]
        if version == 'new':
            all_scores = pd.read_csv(basedir + 'Cluster' + str(cluster) + '.' + celltype + '.updated.snp_scores.tsv', sep='\t', keep_default_na=False)
            all_scores = all_scores[['chr', 'start', 'end', 'rsid',
                                     'effect', 'noneffect', 'ref', 'alt', 
                                     'major', 'minor', 'direction',
                                     'locus_num', 'gwas', 'gwas_pval', 'coloc',
                                     'explain_pval', 'ism_pval', 'delta_pval',
                                     'magnitude_pval', 'prominence_pval', 'confidence']]
            key_pval = 'prominence_pval'
        elif version == 'old':
            all_scores = pd.read_csv(basedir + 'Cluster' + str(cluster) + '.' + celltype + '.snps.csv', sep='\t', keep_default_na=False)
            all_scores = all_scores[['chr', 'start', 'end', 'rsid',
                                 'effect', 'noneffect', 'direction',
                                 'locus_num', 'gwas', 'gwas_pval', 'coloc',
                                 'explain_pval', 'ism_pval', 'delta_pval','confidence']]
            key_pval = 'explain_pval'
        for index, row in all_scores.iterrows():
            if row['rsid'] in snps_dict:
                if cluster not in snps_dict[row['rsid']]['scored_clusters']:
                    snps_dict[row['rsid']]['scored_clusters'].append(cluster)
                if celltype not in snps_dict[row['rsid']]['scored_celltypes']:
                    snps_dict[row['rsid']]['scored_celltypes'].append(celltype)
                if snps_dict[row['rsid']]['effect'] == 'NA' or snps_dict[row['rsid']]['noneffect'] == 'NA':
                    if row['effect'] != 'NA' and row['noneffect'] != 'NA':
                        update_dict(snps_dict, row, cluster, celltype, version)
                    elif row['confidence'] > snps_dict[row['rsid']]['confidence']:
                        update_dict(snps_dict, row, cluster, celltype, version)
                    elif row['confidence'] == snps_dict[row['rsid']]['confidence'] and row[key_pval] < snps_dict[row['rsid']][key_pval]:
                        update_dict(snps_dict, row, cluster, celltype, version)
                elif row['effect'] != 'NA' and row['noneffect'] != 'NA':
                    if row['confidence'] > snps_dict[row['rsid']]['confidence']:
                        update_dict(snps_dict, row, cluster, celltype, version)
                    elif row['confidence'] == snps_dict[row['rsid']]['confidence'] and row[key_pval] < snps_dict[row['rsid']][key_pval]:
                        update_dict(snps_dict, row, cluster, celltype, version)
                    elif row['confidence'] > -1:
                        if cluster not in snps_dict[row['rsid']]['sig_clusters']:
                            snps_dict[row['rsid']]['sig_clusters'].append(cluster)
                        if celltype not in snps_dict[row['rsid']]['sig_celltypes']:
                            snps_dict[row['rsid']]['sig_celltypes'].append(celltype)
            else:
                snps_dict[row['rsid']] = {all_scores.columns[i]:row[all_scores.columns[i]] for i in range(len(all_scores.columns))}
                snps_dict[row['rsid']]['best_cluster'] = ''
                snps_dict[row['rsid']]['best_celltype'] = ''
                snps_dict[row['rsid']]['sig_clusters'] = []
                snps_dict[row['rsid']]['sig_celltypes'] = []
                snps_dict[row['rsid']]['scored_clusters'] = [cluster]
                snps_dict[row['rsid']]['scored_celltypes'] = [celltype]
                update_dict(snps_dict, row, cluster, celltype, version)

## Make DataFrame from Updated Scores

In [16]:
def make_df(basedir, version):
    snps_dict = {}
    if version == 'new':
        populate_dict(basedir, snps_dict, 'new')
    elif version == 'old':
        populate_dict(basedir, snps_dict, 'old')
    pandas_dict = {i:[] for i in snps_dict[list(snps_dict.keys())[0]]}
    for snp in snps_dict:
        for key in snps_dict[snp]:
            pandas_dict[key].append(snps_dict[snp][key])
    merged_scores = pd.DataFrame.from_dict(pandas_dict)
    if version == 'new':
        merged_scores.sort_values(by=['prominence_pval'], inplace=True)
        merged_scores.sort_values(by=['confidence'], ascending=False, inplace=True)
    elif version == 'old':
        merged_scores.sort_values(by=['explain_pval'], inplace=True)
        merged_scores.sort_values(by=['confidence'], ascending=False, inplace=True)
    print('All Scored SNPs:', merged_scores.shape[0])
    print()
    display(merged_scores.head())
    print()
    sig_snps = merged_scores.loc[merged_scores['confidence'] > -1].copy()
    print('Significant SNPs:', sig_snps.shape[0])
    print()
    display(sig_snps.head())
    print()
    high_conf_snps = merged_scores.loc[merged_scores['confidence'] == 2].copy()
    print('High Confidence SNPs:', high_conf_snps.shape[0])
    print()
    display(high_conf_snps.head())
    print()
    medium_conf_snps = merged_scores.loc[merged_scores['confidence'] == 1].copy()
    print('Medium Confidence SNPs:', medium_conf_snps.shape[0])
    print()
    display(medium_conf_snps.head())
    print()
    low_conf_snps = merged_scores.loc[merged_scores['confidence'] == 0].copy()
    print('Low Confidence SNPs:', low_conf_snps.shape[0])
    print()
    display(low_conf_snps.head())
    print()
    return merged_scores, sig_snps, high_conf_snps, medium_conf_snps, low_conf_snps

## Get Updated Scores for SNPs

In [17]:
new_scores, new_sig_snps, new_high_conf_snps, \
new_medium_conf_snps, new_low_conf_snps = make_df('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_snp_scores/', 'new')

All Scored SNPs: 1677



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
8,chr11,60251676,60251677,rs636317,C,T,C,T,T,C,+,11,Alzheimers_Kunkle_2019,5.91e-15,False,0.000032,0.000025,0.000024,0.000000,0.000687,2,20,oligodendrocytes,"[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]","[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]"
374,chr17,45783750,45783751,rs62057073,T,C,C,T,C,T,-,54,23andme_PD_hg38,2.16e-22,True,0.049278,0.044287,0.036197,0.016375,0.039693,2,21,oligodendrocytes,[21],[oligodendrocytes],"[1, 2, 3, 4, 7, 9, 11, 12, 15, 20, 21, 22]","[excitatory_neurons, inhibitory_neurons, unkno..."
802,chr17,4901871,4901872,rs79436576,T,C,C,T,C,T,+,55,Alzheimers_Lambert_2013,4.79e-05,True,0.003710,0.002886,0.002594,0.000690,0.021172,2,13,astrocytes,"[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte...","[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte..."
929,chr17,45947293,45947294,rs62062786,G,C,G,"A,C",G,"A,C",+,54,23andme_PD_hg38,4.53e-22,True,0.000357,0.000188,0.000207,0.019517,0.021981,2,9,opcs,"[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]","[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]"
299,chr6,27772520,27772521,rs760587,C,A,C,A,C,A,-,110,Nalls_23andMe,6.44e-06,False,0.017827,0.014254,0.014910,0.003024,0.024227,2,20,oligodendrocytes,"[19, 20, 21, 22, 23]",[oligodendrocytes],"[1, 2, 3, 8, 10, 11, 12, 13, 14, 15, 16, 19, 2...","[excitatory_neurons, inhibitory_neurons, opcs,..."



Significant SNPs: 128



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
8,chr11,60251676,60251677,rs636317,C,T,C,T,T,C,+,11,Alzheimers_Kunkle_2019,5.91e-15,False,0.000032,0.000025,0.000024,0.000000,0.000687,2,20,oligodendrocytes,"[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]","[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]"
374,chr17,45783750,45783751,rs62057073,T,C,C,T,C,T,-,54,23andme_PD_hg38,2.16e-22,True,0.049278,0.044287,0.036197,0.016375,0.039693,2,21,oligodendrocytes,[21],[oligodendrocytes],"[1, 2, 3, 4, 7, 9, 11, 12, 15, 20, 21, 22]","[excitatory_neurons, inhibitory_neurons, unkno..."
802,chr17,4901871,4901872,rs79436576,T,C,C,T,C,T,+,55,Alzheimers_Lambert_2013,4.79e-05,True,0.003710,0.002886,0.002594,0.000690,0.021172,2,13,astrocytes,"[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte...","[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte..."
929,chr17,45947293,45947294,rs62062786,G,C,G,"A,C",G,"A,C",+,54,23andme_PD_hg38,4.53e-22,True,0.000357,0.000188,0.000207,0.019517,0.021981,2,9,opcs,"[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]","[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]"
299,chr6,27772520,27772521,rs760587,C,A,C,A,C,A,-,110,Nalls_23andMe,6.44e-06,False,0.017827,0.014254,0.014910,0.003024,0.024227,2,20,oligodendrocytes,"[19, 20, 21, 22, 23]",[oligodendrocytes],"[1, 2, 3, 8, 10, 11, 12, 13, 14, 15, 16, 19, 2...","[excitatory_neurons, inhibitory_neurons, opcs,..."



High Confidence SNPs: 40



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
8,chr11,60251676,60251677,rs636317,C,T,C,T,T,C,+,11,Alzheimers_Kunkle_2019,5.91e-15,False,0.000032,0.000025,0.000024,0.000000,0.000687,2,20,oligodendrocytes,"[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]","[1, 4, 19, 20, 24]","[excitatory_neurons, oligodendrocytes, microglia]"
374,chr17,45783750,45783751,rs62057073,T,C,C,T,C,T,-,54,23andme_PD_hg38,2.16e-22,True,0.049278,0.044287,0.036197,0.016375,0.039693,2,21,oligodendrocytes,[21],[oligodendrocytes],"[1, 2, 3, 4, 7, 9, 11, 12, 15, 20, 21, 22]","[excitatory_neurons, inhibitory_neurons, unkno..."
802,chr17,4901871,4901872,rs79436576,T,C,C,T,C,T,+,55,Alzheimers_Lambert_2013,4.79e-05,True,0.003710,0.002886,0.002594,0.000690,0.021172,2,13,astrocytes,"[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte...","[6, 11, 13, 14, 15, 19, 20, 21, 22, 24]","[nigral_neurons, inhibitory_neurons, astrocyte..."
929,chr17,45947293,45947294,rs62062786,G,C,G,"A,C",G,"A,C",+,54,23andme_PD_hg38,4.53e-22,True,0.000357,0.000188,0.000207,0.019517,0.021981,2,9,opcs,"[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]","[9, 10, 15, 16, 20, 22]","[opcs, astrocytes, oligodendrocytes]"
299,chr6,27772520,27772521,rs760587,C,A,C,A,C,A,-,110,Nalls_23andMe,6.44e-06,False,0.017827,0.014254,0.014910,0.003024,0.024227,2,20,oligodendrocytes,"[19, 20, 21, 22, 23]",[oligodendrocytes],"[1, 2, 3, 8, 10, 11, 12, 13, 14, 15, 16, 19, 2...","[excitatory_neurons, inhibitory_neurons, opcs,..."



Medium Confidence SNPs: 31



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
1193,chr17,45955512,45955513,rs78555354,G,A,A,G,A,G,-,54,23andme_PD_hg38,3.22e-22,True,0.048234,0.040545,0.040822,0.053514,0.099351,1,17,astrocytes,[17],[astrocytes],"[14, 15, 17]",[astrocytes]
329,chr11,86069405,86069406,rs3016327,T,C,T,C,C,T,-,16,Alzheimers_Jansen_2018,1.41e-11,False,0.023854,0.014920,0.013453,0.037161,0.154323,1,12,inhibitory_neurons,[12],[inhibitory_neurons],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[excitatory_neurons, inhibitory_neurons, nigra..."
10,chr10,102245652,102245653,rs2296887,T,C,T,C,T,C,-,1,Nalls_23andMe,0.00697732,False,0.016220,0.013148,0.013543,0.006184,0.058309,1,4,excitatory_neurons,"[1, 2, 4, 11]","[excitatory_neurons, inhibitory_neurons]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[excitatory_neurons, inhibitory_neurons, nigra..."
341,chr4,959909,959910,rs4690326,C,A,A,C,C,A,-,102,23andme_PD_hg38,3.5299999999999997e-10,True,0.021397,0.016705,0.015900,0.060837,0.058935,1,22,oligodendrocytes,[22],[oligodendrocytes],"[1, 4, 20, 21, 22]","[excitatory_neurons, oligodendrocytes]"
1327,chr17,45825577,45825578,rs1912151,T,C,C,T,C,T,-,54,23andme_PD_hg38,1.33e-22,True,0.049351,0.040850,0.040896,0.085247,0.060312,1,19,oligodendrocytes,[19],[oligodendrocytes],"[19, 20, 21, 22, 23]",[oligodendrocytes]



Low Confidence SNPs: 57



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
1382,chr17,45882956,45882957,rs56365901,G,A,A,G,A,G,-,54,23andme_PD_hg38,4.42e-22,True,0.005790,0.005021,0.004366,0.262776,0.358745,0,22,oligodendrocytes,"[20, 21, 22]",[oligodendrocytes],"[20, 21, 22]",[oligodendrocytes]
569,chr11,47339532,47339533,rs2856653,T,C,T,C,C,T,+,9,Alzheimers_Kunkle_2019,6.19e-08,True,0.027840,0.022575,0.021415,0.054908,0.119754,0,2,inhibitory_neurons,[2],[inhibitory_neurons],[2],[inhibitory_neurons]
1085,chr12,40019502,40019503,rs28370664,C,A,C,A,C,A,-,25,23andme_PD_hg38,1.75e-20,False,0.047485,0.039387,0.039307,0.253662,0.387948,0,19,oligodendrocytes,[19],[oligodendrocytes],"[12, 19]","[inhibitory_neurons, oligodendrocytes]"
933,chr17,4908319,4908320,rs7214776,C,T,T,C,T,C,+,55,Alzheimers_Lambert_2013,4.270000000000001e-05,True,0.007311,0.005403,0.005572,0.254589,0.206522,0,9,opcs,"[9, 14, 15]","[opcs, astrocytes]","[9, 14, 15]","[opcs, astrocytes]"
98,chr3,122416407,122416408,rs55911744,T,G,T,G,T,G,-,86,Nalls_23andMe,0.000499,False,0.021800,0.017305,0.016720,0.170526,0.255987,0,16,astrocytes,"[9, 13, 15, 16, 19, 22, 23, 24]","[opcs, astrocytes, oligodendrocytes, microglia]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[excitatory_neurons, inhibitory_neurons, nigra..."


## Get Old Scores for SNPs

In [18]:
old_scores, old_sig_snps, old_high_conf_snps, \
old_medium_conf_snps, old_low_conf_snps = make_df('/mnt/lab_data3/soumyak/adpd/sig_snps/', 'old')

All Scored SNPs: 123



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
19,chr12,33530906,33530907,rs935534,G,A,-,23,23andme_PD_hg38,1.460670e-07,False,0.001517,0.003705,0.004138,2,2,inhibitory_neurons,[2],[inhibitory_neurons],[2],[inhibitory_neurons]
37,chr17,45959569,45959570,rs62063279,G,A,+,54,23andme_PD_hg38,1.212960e-21,True,0.024348,0.023804,0.025121,2,4,excitatory_neurons,[4],[excitatory_neurons],[4],[excitatory_neurons]
88,chr17,75088242,75088243,rs55827552,A,G,-,61,Alzheimers_Kunkle_2019,1.869000e-03,True,0.028007,0.038755,0.038833,2,18,doublets,[18],[doublets],[18],[doublets]
82,chr17,45973461,45973462,rs9896485,G,C,-,54,23andme_PD_hg38,7.790110e-09,True,0.028134,0.024579,0.025640,2,16,astrocytes,[16],[astrocytes],[16],[astrocytes]
32,chr17,45701984,45701985,rs17688922,A,G,-,54,23andme_PD_hg38,2.054400e-22,True,0.028386,0.040435,0.043605,2,3,excitatory_neurons,[3],[excitatory_neurons],[3],[excitatory_neurons]



Significant SNPs: 123



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
19,chr12,33530906,33530907,rs935534,G,A,-,23,23andme_PD_hg38,1.460670e-07,False,0.001517,0.003705,0.004138,2,2,inhibitory_neurons,[2],[inhibitory_neurons],[2],[inhibitory_neurons]
37,chr17,45959569,45959570,rs62063279,G,A,+,54,23andme_PD_hg38,1.212960e-21,True,0.024348,0.023804,0.025121,2,4,excitatory_neurons,[4],[excitatory_neurons],[4],[excitatory_neurons]
88,chr17,75088242,75088243,rs55827552,A,G,-,61,Alzheimers_Kunkle_2019,1.869000e-03,True,0.028007,0.038755,0.038833,2,18,doublets,[18],[doublets],[18],[doublets]
82,chr17,45973461,45973462,rs9896485,G,C,-,54,23andme_PD_hg38,7.790110e-09,True,0.028134,0.024579,0.025640,2,16,astrocytes,[16],[astrocytes],[16],[astrocytes]
32,chr17,45701984,45701985,rs17688922,A,G,-,54,23andme_PD_hg38,2.054400e-22,True,0.028386,0.040435,0.043605,2,3,excitatory_neurons,[3],[excitatory_neurons],[3],[excitatory_neurons]



High Confidence SNPs: 44



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
19,chr12,33530906,33530907,rs935534,G,A,-,23,23andme_PD_hg38,1.460670e-07,False,0.001517,0.003705,0.004138,2,2,inhibitory_neurons,[2],[inhibitory_neurons],[2],[inhibitory_neurons]
37,chr17,45959569,45959570,rs62063279,G,A,+,54,23andme_PD_hg38,1.212960e-21,True,0.024348,0.023804,0.025121,2,4,excitatory_neurons,[4],[excitatory_neurons],[4],[excitatory_neurons]
88,chr17,75088242,75088243,rs55827552,A,G,-,61,Alzheimers_Kunkle_2019,1.869000e-03,True,0.028007,0.038755,0.038833,2,18,doublets,[18],[doublets],[18],[doublets]
82,chr17,45973461,45973462,rs9896485,G,C,-,54,23andme_PD_hg38,7.790110e-09,True,0.028134,0.024579,0.025640,2,16,astrocytes,[16],[astrocytes],[16],[astrocytes]
32,chr17,45701984,45701985,rs17688922,A,G,-,54,23andme_PD_hg38,2.054400e-22,True,0.028386,0.040435,0.043605,2,3,excitatory_neurons,[3],[excitatory_neurons],[3],[excitatory_neurons]



Medium Confidence SNPs: 40



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
34,chr2,134718389,134718390,rs62171393,G,C,-,78,Nalls_23andMe,9.020940e-03,False,0.144674,0.047058,0.047824,1,3,excitatory_neurons,[3],[excitatory_neurons],[3],[excitatory_neurons]
83,chr19,45137506,45137507,rs536518226,C,G,+,71,Alzheimers_Jansen_2018,1.882816e-19,True,0.116502,0.032512,0.032970,1,16,astrocytes,[16],[astrocytes],[16],[astrocytes]
54,chr7,100217743,100217744,rs1727139,C,T,-,117,Alzheimers_Jansen_2018,9.848070e-10,True,0.120150,0.039411,0.038600,1,9,opcs,[9],[opcs],[9],[opcs]
119,chr14,92466230,92466231,rs10130373,C,A,+,39,Alzheimers_Jansen_2018,6.157006e-09,False,0.121176,0.007148,0.007459,1,24,microglia,[24],[microglia],[24],[microglia]
86,chr17,45955512,45955513,rs78555354,G,A,-,54,23andme_PD_hg38,3.219870e-22,True,0.122640,0.029784,0.032418,1,17,astrocytes,[17],[astrocytes],[17],[astrocytes]



Low Confidence SNPs: 39



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
87,chr3,52190572,52190573,rs181190702,C,A,-,93,Chang_23andMe_Parkinsons,1.329290e-03,False,0.999998,0.045065,0.047698,0,17,astrocytes,[17],[astrocytes],[17],[astrocytes]
36,chr19,44915228,44915229,rs12691088,A,G,+,71,Alzheimers_Kunkle_2019,2.690000e-238,True,0.999996,0.031232,0.032746,0,11,inhibitory_neurons,"[3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[excitatory_neurons, nigral_neurons, opcs, inh...","[3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[excitatory_neurons, nigral_neurons, opcs, inh..."
15,chr3,52237762,52237763,rs146527642,G,C,-,93,Chang_23andMe_Parkinsons,9.369830e-04,False,0.999997,0.026073,0.028679,0,1,excitatory_neurons,[1],[excitatory_neurons],[1],[excitatory_neurons]
16,chr17,45965572,45965573,rs112003311,G,A,+,54,23andme_PD_hg38,2.301910e-22,True,0.999997,0.023177,0.027944,0,1,excitatory_neurons,"[1, 3]",[excitatory_neurons],"[1, 3]",[excitatory_neurons]
17,chr1,161185601,161185602,rs4575098,A,G,+,12,Alzheimers_Jansen_2018,2.051494e-10,True,0.999997,0.039539,0.045172,0,1,excitatory_neurons,"[1, 4]",[excitatory_neurons],"[1, 4]",[excitatory_neurons]


## Find Significant SNPs that were gained or lost

In [19]:
lost_sig_snps = []
gained_sig_snps = []
for ind,rw in old_scores.iterrows():
    found = False
    for index,row in new_scores.iterrows():
        if rw['rsid'] == row['rsid']:
            found = True
            if row['confidence'] == -1:
                lost_sig_snps.append(rw['rsid'])
    assert found == True
for index,row in new_scores.iterrows():
    if row['confidence'] > -1:
        found = False
        for ind,rw in old_scores.iterrows():
            if row['rsid'] == rw['rsid']:
                found = True
        if found == False:
            gained_sig_snps.append(row['rsid'])

print('Lost Significant SNPs: ', lost_sig_snps)
print('Count: ', len(lost_sig_snps))
print('Gained Significant SNPs: ', gained_sig_snps)
print('Count: ', len(gained_sig_snps))

Lost Significant SNPs:  ['rs4802235', 'rs62057150', 'rs755934', 'rs181190702', 'rs2270425', 'rs1532276']
Count:  6
Gained Significant SNPs:  ['rs893433', 'rs62063842', 'rs3772034', 'rs1912151', 'rs59587437', 'rs62054439', 'rs10425982', 'rs34746918', 'rs4854244', 'rs12493578', 'rs62060787']
Count:  11


## Compare SNPs between old and new scoring methods

In [20]:
rsid = 'rs1912151'
print('Old Score:')
print()
display(old_scores.loc[old_scores['rsid'] == rsid])
print()
print('New Score:')
print()
display(new_scores.loc[new_scores['rsid'] == rsid])
print()

Old Score:



,chr,start,end,rsid,effect,noneffect,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes



New Score:



,chr,start,end,rsid,effect,noneffect,ref,alt,major,minor,direction,locus_num,gwas,gwas_pval,coloc,explain_pval,ism_pval,delta_pval,magnitude_pval,prominence_pval,confidence,best_cluster,best_celltype,sig_clusters,sig_celltypes,scored_clusters,scored_celltypes
1327,chr17,45825577,45825578,rs1912151,T,C,C,T,C,T,-,54,23andme_PD_hg38,1.33e-22,True,0.049351,0.04085,0.040896,0.085247,0.060312,1,19,oligodendrocytes,[19],[oligodendrocytes],"[19, 20, 21, 22, 23]",[oligodendrocytes]


## Save merged SNP scores table

In [21]:
new_scores.to_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_snp_scores/all.unique.snps.scores.tsv', sep='\t', index=False)
new_sig_snps.to_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_sig_snps/all.unique.sig.snps.scores.tsv', sep='\t', index=False)

## Create New Cluster Summary Table

In [32]:
summary_dict = {'cluster': ['Cluster'+str(i) for i in range(1,25)],
                'celltype': [clust_to_celltype[i] for i in range(1,25)],
                'scored': [], 'significant': [], 'high_confidence': [],
                'medium_confidence': [], 'low_confidence': []}
                
for cluster in range(1, 25):
    scored = 0
    significant = 0
    high = 0
    medium = 0
    low = 0
    for index,row in new_scores.iterrows():
        if cluster in row['scored_clusters']:
            scored += 1
        if cluster in row['sig_clusters']:
            #print(row['rsid'])
            #print(row['effect'])
            significant += 1
    sig_snps = pd.read_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_sig_snps/Cluster'
                           + str(cluster) + '.' + clust_to_celltype[cluster] + '.updated.unique.sig_snps.tsv', sep='\t', keep_default_na=False)
    #print(len(sig_snps))
    #print(significant)
    assert len(sig_snps) == significant
    high = len(sig_snps.loc[sig_snps['confidence'] == 2])
    medium = len(sig_snps.loc[sig_snps['confidence'] == 1])
    low = len(sig_snps.loc[sig_snps['confidence'] == 0])
    summary_dict['scored'].append(scored)
    summary_dict['significant'].append(significant)
    summary_dict['high_confidence'].append(high)
    summary_dict['medium_confidence'].append(medium)
    summary_dict['low_confidence'].append(low)
    
summary_dict['cluster'].append('All')
summary_dict['celltype'].append('All')
summary_dict['scored'].append(len(new_scores))
summary_dict['significant'].append(len(new_sig_snps))
summary_dict['high_confidence'].append(len(new_high_conf_snps))
summary_dict['medium_confidence'].append(len(new_medium_conf_snps))
summary_dict['low_confidence'].append(len(new_low_conf_snps))
    
summary_df = pd.DataFrame.from_dict(summary_dict)
display(summary_df)
print()
print('Shape: ', summary_df.shape)
summary_df.to_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_snp_scores/snp_score_summary_by_cluster.tsv', sep='\t', index=False)

,cluster,celltype,scored,significant,high_confidence,medium_confidence,low_confidence
0,Cluster1,excitatory_neurons,567,23,8,7,8
1,Cluster2,inhibitory_neurons,456,19,5,7,7
2,Cluster3,excitatory_neurons,356,13,3,3,7
3,Cluster4,excitatory_neurons,391,19,3,7,9
4,Cluster5,nigral_neurons,175,6,2,1,3
5,Cluster6,nigral_neurons,221,13,5,3,5
6,Cluster7,unknown_neurons,339,12,3,7,2
7,Cluster8,opcs,183,5,2,2,1
8,Cluster9,opcs,397,22,6,4,12
9,Cluster10,opcs,209,9,3,1,5



Shape:  (25, 7)


In [33]:
old_summary_dict = {'cluster': ['Cluster'+str(i) for i in range(1,25)],
                'celltype': [clust_to_celltype[i] for i in range(1,25)],
                'scored': summary_dict['scored'], 'significant': [], 'high_confidence': [],
                'medium_confidence': [], 'low_confidence': []}
                
for cluster in range(1, 25):
    significant = 0
    high = 0
    medium = 0
    low = 0
    for index,row in old_scores.iterrows():
        if cluster in row['sig_clusters']:
            #print(row['rsid'])
            #print(row['effect'])
            significant += 1
    sig_snps = pd.read_csv('/mnt/lab_data3/soumyak/adpd/sig_snps/Cluster' + str(cluster) + '.'
                           + clust_to_celltype[cluster] + '.snps.csv', sep='\t', keep_default_na=False)
    #print(len(sig_snps))
    #print(significant)
    assert len(sig_snps) == significant
    high = len(sig_snps.loc[sig_snps['confidence'] == 2])
    medium = len(sig_snps.loc[sig_snps['confidence'] == 1])
    low = len(sig_snps.loc[sig_snps['confidence'] == 0])
    old_summary_dict['significant'].append(significant)
    old_summary_dict['high_confidence'].append(high)
    old_summary_dict['medium_confidence'].append(medium)
    old_summary_dict['low_confidence'].append(low)
    
old_summary_dict['cluster'].append('All')
old_summary_dict['celltype'].append('All')
old_summary_dict['significant'].append(len(old_sig_snps))
old_summary_dict['high_confidence'].append(len(old_high_conf_snps))
old_summary_dict['medium_confidence'].append(len(old_medium_conf_snps))
old_summary_dict['low_confidence'].append(len(old_low_conf_snps))
    
old_summary_df = pd.DataFrame.from_dict(old_summary_dict)
display(old_summary_df)
print()
print('Shape: ', old_summary_df.shape)

,cluster,celltype,scored,significant,high_confidence,medium_confidence,low_confidence
0,Cluster1,excitatory_neurons,567,19,8,4,7
1,Cluster2,inhibitory_neurons,456,20,8,6,6
2,Cluster3,excitatory_neurons,356,13,5,3,5
3,Cluster4,excitatory_neurons,391,15,6,3,6
4,Cluster5,nigral_neurons,175,6,2,1,3
5,Cluster6,nigral_neurons,221,10,4,2,4
6,Cluster7,unknown_neurons,339,14,5,6,3
7,Cluster8,opcs,183,6,3,1,2
8,Cluster9,opcs,397,22,9,5,8
9,Cluster10,opcs,209,8,3,2,3



Shape:  (25, 7)
